In [1]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.6 MB 5.1 MB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 
     |████████████████████████████████| 213 kB 48.5 MB/s 
     |████████████████████████████████| 84 kB 3.4 MB/s 
     |████████████████████████████████| 56 kB 3.4 MB/s 
     |████████████████████████████████| 2.3 MB 36.0 MB/s 
     |████████████████████████████████| 272 kB 44.0 MB/s 
     |████████████████████████████████| 84 kB 3.3 MB/s 
     |████████████████████████████████| 106 kB 46.8 MB/s 
     |████████████████████████████████| 55 kB 3.2 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 80 kB 6.6 MB/s 
     |████████████████████████████████| 68 kB 4.7 MB/s 
     |████████████████████████████████| 46 kB 3.3 MB/s 
     |████████████████████████████████| 4.1 MB 41.0 MB/s 
     |████████████████████████████████| 856 kB 45

# Few Shot App

In [11]:
!pip install -U tensorflow-addons
import io
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
from tensorflow.keras import layers
from keras.regularizers import l2
from tensorflow.python.framework.ops import re
import json
import requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import tensorflow_hub as hub
model_url = "https://tfhub.dev/google/bit/m-r101x1/1"
module = hub.KerasLayer(model_url)

In [6]:
from tensorflow.python.framework.ops import re
model = tf.keras.Sequential([
    module,
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dropout(0.4),
    # tf.keras.layers.Dropout(0.2),
    # tf.keras.layers.Dense(64, activation="relu"),
    # tf.keras.layers.Dropout(0.2),
    # tf.keras.layers.Dense(64, activation="relu"),
    # tf.keras.layers.Dropout(0.2),
    # tf.keras.layers.Dense(32, activation="relu"),
    # tf.keras.layers.Dropout(0.2),
    # tf.keras.layers.Dense(8, activation="relu"),
    # tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
    tf.keras.layers.Dense(128, activation=None), # No activation on final dense layer
    tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings
])

In [22]:
batch_size=256
image_size=224
resize_and_rescale = tf.keras.Sequential([
    layers.Rescaling(1./255),
    layers.Resizing(image_size, image_size)
])

In [12]:

model.build(input_shape=(batch_size,image_size,image_size,3))
model.summary()
model.load_weights('assets/TripletLoss_bestModel.hdf5')

def img_to_encoding(img, model):
    x_train = np.expand_dims(img, axis=0)
    embedding = model.predict_on_batch(x_train)
    return embedding

# apping the number label to actual names of the flowers
file_location='https://github.com/CheloGE/ML-tensorflow-Image_classifier-oxford_flowers102/blob/master/label_map.json?raw=1'
r = requests.get(file_location, stream=True)
class_names=json.loads(r.content.decode('utf-8'))
class_names = {int(k):v for k,v in class_names.items()}

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (256, 2048)               42492480  
                                                                 
 dropout_2 (Dropout)         (256, 2048)               0         
                                                                 
 dense_2 (Dense)             (256, 256)                524544    
                                                                 
 dropout_3 (Dropout)         (256, 256)                0         
                                                                 
 dense_3 (Dense)             (256, 128)                32896     
                                                                 
 lambda_1 (Lambda)           (256, 128)                0         
                                                                 
Total params: 43,049,920
Trainable params: 557,440
Non

In [14]:
import pickle

database = pickle.load(open("assets/database_r101.p", "rb"))

In [27]:
def which_flower(img, database, model):
    
    ## Step 1: Compute the target "encoding" for the image. Use img_to_encoding() see example above. ## (≈ 1 line)
    encoding = img_to_encoding(img,model)
    
    ## Step 2: Find the closest encoding ##
    # Initialize "min_dist" to a large value, say 100 (≈1 line)
    min_dist = 100
    
    #Loop over the database dictionary's names and encodings.
    for (name, db_enc) in database.items():
        
        # Compute L2 distance between the target "encoding" and the current db_enc from the database. (≈ 1 line)
        dist = np.linalg.norm(encoding - db_enc)

        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name. (≈ 3 lines)
        if dist < min_dist:
            min_dist = dist
            identity = name
    
    # YOUR CODE ENDS HERE
    
    if min_dist > 0.7:
        print("Not in the database.")
        return (0,-1)
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))
        
    return min_dist, identity

In [ ]:
import gradio as gr
import torch
import requests

In [31]:



def predict(inp):
  # inp = transforms.ToTensor()(inp).unsqueeze(0)
  image =  tf.convert_to_tensor(inp)
  image = resize_and_rescale(image)
  print(image.shape)
  pred = which_flower(image,database,model)
  # with torch.no_grad():
  #   prediction = torch.nn.functional.softmax(model(inp)[0], dim=0)
  #   confidences = {labels[i]: float(prediction[i]) for i in range(1000)}    
  print(pred[1])
  return pred[1]

demo = gr.Interface(fn=predict, 
             inputs=gr.Image(shape=(224, 224)),
             outputs=gr.outputs.Label(num_top_classes=1),
             )
             
demo.launch(debug=True,share=True)

/usr/local/lib/python3.7/dist-packages/gradio/outputs.py:197: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  "Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components",
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://b75f0fed705d1032.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


(224, 224, 3)
it's hibiscus, the distance is 0.17538403
hibiscus
Keyboard interruption in main thread... closing server.


(<gradio.routes.App at 0x7ff622c37450>,
 'http://127.0.0.1:7867/',
 'https://b75f0fed705d1032.gradio.app')